In [21]:
#  <img src="./logo_transparent.png" width="70"> Step1 - Crawling des datas du championnat <img src="./logoffhb.png" width="100"> 

In [1]:
import geopandas as gpd
import sqlite3
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import folium
import time
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
pd.set_option('display.max_rows', None) # Show all raws from pandas df

### Loading match address from Championnat U18

In [2]:
conn = sqlite3.connect('../ffhb_v2023/Championnat_U18_GrandEst.sqlite')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM Matchs", conn)
conn.close()

In [3]:
Equipe='EPINAL U18M'
df2 = df[(df['equipe_dom']==Equipe) | (df['equipe_ext']==Equipe)][['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']]
df2

,Journee,Periode,Date,equipe_dom,score_dom,score_ext,equipe_ext,lieu,arbitres
5,1,17/09 AU 18/09/2022,samedi 17 septembre 16H00,COLMAR HC U18M,31,20,EPINAL U18M,MONTAGNE VERTE PLACE DE LA MONTAGNE VERTE COLMAR,— HIRTH SEBASTIEN
8,2,24/09 AU 25/09/2022,samedi 24 septembre 16H00,BASSIN MUSSIPONTAIN U18M,25,18,EPINAL U18M,CHARLES ROTH RUE JACQUES ANQUETIL DIEULOUARD,BENAMARA ABDESLAM —
12,3,01/10 AU 02/10/2022,samedi 1 octobre 16H00,EPINAL U18M,23,21,THIONVILLE U18M,PALAIS DES SPORTS AVENUE DE SAINT DIE EPINAL,LOPES JOSE —
18,4,08/10 AU 09/10/2022,samedi 8 octobre 16H00,EPINAL U18M,26,31,STRASBOURG&SCHILTIGHEIM U18M2,PALAIS DES SPORTS AVENUE DE SAINT DIE EPINAL,LIROT MAXIME —
29,5,15/10 AU 16/10/2022,samedi 15 octobre 18H30,NANCY U18M2,20,23,EPINAL U18M,GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY,— —
32,6,22/10 AU 23/10/2022,samedi 22 octobre 18H00,EPINAL U18M,19,25,METZ U18M,PALAIS DES SPORTS AVENUE DE SAINT DIE EPINAL,SCHUBNEL MICKAEL —
39,7,19/11 AU 20/11/2022,samedi 19 novembre 18H30,ENT MULHOUSE ACADEMIE U18M2,32,23,EPINAL U18M,ANDRE ERBLAND 50 BD PRESIDENT ROOSEVELT MULHOUSE,LIROT MAXIME NOEL ARTHUR
47,8,26/11 AU 27/11/2022,samedi 26 novembre 14H00,EPINAL U18M,28,22,ENT RIED'HANDBALL U18M,PALAIS DES SPORTS AVENUE DE SAINT DIE EPINAL,VILLEMIN ERIC —
52,10,14/01 AU 15/01/2023,samedi 14 janvier 16H00,EPINAL U18M,28,25,LINGOLSHEIM,PALAIS DES SPORTS AVENUE DE SAINT DIE EPINAL,KUZU METIN —
58,11,21/01 AU 22/01/2023,samedi 21 janvier 15H45,SELESTAT U18M2,,,EPINAL U18M,CENTRE SPORTIF INTERCOMMUNAL EUGENE GRIESMAR ...,DOPPLER LUCAS —


In [4]:
df[['Journee','Date','equipe_dom','equipe_ext','lieu']]

,Journee,Date,equipe_dom,equipe_ext,lieu
0,1,samedi 17 septembre 19H00,ENT RIED'HANDBALL U18M,STRASBOURG&SCHILTIGHEIM U18M2,SALLE DU COLLEGE ROMAIN ROLLAND RUE ERCKMANN ...
1,1,samedi 17 septembre 17H30,THIONVILLE U18M,BASSIN MUSSIPONTAIN U18M,GYMNASE JEAN-PIERRE ADAMS RUE SAINT EXUPERY T...
2,1,dimanche 18 septembre 14H00,ENT MULHOUSE ACADEMIE U18M2,HOCHFELDEN&DETTWILLER U18M,ANDRE ERBLAND 50 BD PRESIDENT ROOSEVELT MULHOUSE
3,1,dimanche 18 septembre 14H00,METZ U18M,LINGOLSHEIM,GYMNASE DU TECHNOPOLE 5 BOULEVARD ARAGO METZ
4,1,dimanche 18 septembre 14H00,NANCY U18M2,SELESTAT U18M2,GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY
5,1,samedi 17 septembre 16H00,COLMAR HC U18M,EPINAL U18M,MONTAGNE VERTE PLACE DE LA MONTAGNE VERTE COLMAR
6,2,samedi 24 septembre 18H00,STRASBOURG&SCHILTIGHEIM U18M2,THIONVILLE U18M,GYMNASE DES MALTERIES 49 RUE DES MALTERIES SC...
7,2,samedi 24 septembre 16H30,HOCHFELDEN&DETTWILLER U18M,ENT RIED'HANDBALL U18M,COMPLEXE SPORTIF DE HOCHFELDEN 1 RUE DES 4 VE...
8,2,samedi 24 septembre 16H00,BASSIN MUSSIPONTAIN U18M,EPINAL U18M,CHARLES ROTH RUE JACQUES ANQUETIL DIEULOUARD
9,2,dimanche 25 septembre 16H00,LINGOLSHEIM,ENT MULHOUSE ACADEMIE U18M2,GYMNASE COLETTE BESSON 30 MARIA CALAS LINGOLSHEIM


In [5]:
# Liste de tous les gymnases
adresses = list(df['lieu'].unique())
adresses

['SALLE DU COLLEGE ROMAIN ROLLAND  RUE ERCKMANN CHATRIAN ERSTEIN',
 'GYMNASE JEAN-PIERRE ADAMS  RUE SAINT EXUPERY THIONVILLE',
 'ANDRE ERBLAND 50  BD PRESIDENT ROOSEVELT MULHOUSE',
 'GYMNASE DU TECHNOPOLE 5  BOULEVARD ARAGO METZ',
 'GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY',
 'MONTAGNE VERTE  PLACE DE LA MONTAGNE VERTE COLMAR',
 'GYMNASE DES MALTERIES 49  RUE DES MALTERIES SCHILTIGHEIM',
 'COMPLEXE SPORTIF DE HOCHFELDEN 1  RUE DES 4 VENTS HOCHFELDEN',
 'CHARLES ROTH  RUE JACQUES ANQUETIL DIEULOUARD',
 'GYMNASE COLETTE BESSON 30 MARIA CALAS LINGOLSHEIM',
 'COSEC EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT',
 'PALAIS DES SPORTS  AVENUE DE SAINT DIE EPINAL',
 "HALL DES SPORTS  RUE D'ETTENHEIM BENFELD",
 'COMPLEXE SPORTIF  RUE DE LA FORET NOIRE RIXHEIM',
 'CENTRE SPORTIF INTERCOMMUNAL EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT',
 "GYMNASE DE L'EUROPE  ALLES D'ATHENES SCHILTIGHEIM",
 '— — —',
 'GYMNASE DE LA DOLLER  DE TOULON MULHOUSE']

# Extraction Manuelle

In [6]:
# Gymnases_2023 = {add:('lat','lon','equipe') for add in adresses}
# Gymnases_2023

In [17]:
Gymnases_2023 = {'SALLE DU COLLEGE ROMAIN ROLLAND  RUE ERCKMANN CHATRIAN ERSTEIN': (48.4223835597174, 7.651929290215314,  "ENT RIED'HANDBALL U18M"),
                 'GYMNASE DU TECHNOPOLE 5  BOULEVARD ARAGO METZ': (49.10206046306156, 6.221777042356304,'METZ U18M'),
                 'GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY':(48.676969018079625, 6.193471596314589,'NANCY U18M2'),
                 'GYMNASE JEAN-PIERRE ADAMS  RUE SAINT EXUPERY THIONVILLE': (49.36877802825876, 6.1617871693498225,  'THIONVILLE U18M'),
                 'ANDRE ERBLAND 50  BD PRESIDENT ROOSEVELT MULHOUSE': (47.753907195499984, 7.3282587269733535, 'ENT MULHOUSE ACADEMIE U18M2'),
                 'PALAIS DES SPORTS  AVENUE DE SAINT DIE EPINAL': (48.1866131980697, 6.470802440478821,'EPINAL U18M'),
                 'GYMNASE DES MALTERIES 49  RUE DES MALTERIES SCHILTIGHEIM': (48.60461444093289, 7.743636698162694,'STRASBOURG&SCHILTIGHEIM U18M2'),
                 'COMPLEXE SPORTIF DE HOCHFELDEN 1  RUE DES 4 VENTS HOCHFELDEN':(48.761002955531666, 7.564414127393387,'HOCHFELDEN&DETTWILLER U18M'), 
                 'CHARLES ROTH  RUE JACQUES ANQUETIL DIEULOUARD': (48.84118446985741, 6.059275127005468,'BASSIN MUSSIPONTAIN U18M'),
                 'MONTAGNE VERTE  PLACE DE LA MONTAGNE VERTE COLMAR': (48.075639122459386, 7.360304426982781,'COLMAR HC U18M'),
                 'COSEC EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT': (48.25196381266806, 7.4510275133350135,'SELESTAT U18M2'),
                 'COMPLEXE SPORTIF  RUE DE LA FORET NOIRE RIXHEIM': (47.76173567627065, 7.395315665557758,'ENT MULHOUSE ACADEMIE U18M2'),
                 'GYMNASE COLETTE BESSON 30 MARIA CALAS LINGOLSHEIM':(48.5592291985453, 7.700333996311093,'LINGOLSHEIM'),
                 "HALL DES SPORTS  RUE D'ETTENHEIM BENFELD":(48.371307612333005, 7.603938933504872,"ENT RIED'HANDBALL U18M"),
                 'CENTRE SPORTIF INTERCOMMUNAL EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT':(48.25190245946577, 7.450157263429608,'SELESTAT U18M2'),
                 "GYMNASE DE L'EUROPE  ALLES D'ATHENES SCHILTIGHEIM":(48.6053022831414, 7.707964297439742,'STRASBOURG&SCHILTIGHEIM U18M2'),
                 'GYMNASE DE LA DOLLER  DE TOULON MULHOUSE':(47.77417716916886, 7.333386903586349,'ENT MULHOUSE ACADEMIE U18M2')
                }

In [18]:
Gym = [k for k,l in Gymnases_2023.items()]
Gymnases_à_ajouter = [g for g in adresses if g not in Gym]
Gymnases_à_ajouter

['— — —']

In [19]:
df['equipe_dom'].unique()

array(["ENT RIED'HANDBALL U18M", 'THIONVILLE U18M',
       'ENT MULHOUSE ACADEMIE U18M2', 'METZ U18M', 'NANCY U18M2',
       'COLMAR HC U18M', 'STRASBOURG&SCHILTIGHEIM U18M2',
       'HOCHFELDEN&DETTWILLER U18M', 'BASSIN MUSSIPONTAIN U18M',
       'LINGOLSHEIM', 'SELESTAT U18M2', 'EPINAL U18M'], dtype=object)

In [20]:
Gymnases_2022 ={'COSEC EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT': (48.25196381266806, 7.4510275133350135,'SELESTAT'),
 'MONTAGNE VERTE  PLACE DE LA MONTAGNE VERTE COLMAR': (48.075639122459386, 7.360304426982781,'COLMAR'),
 'MARCEL MARTIN 5 RUE USSON DE POITOU FOLSCHVILLER': (49.07691719956764, 6.685560552418234,'FOLSCHWILLER'),
 'COSEC DE RIXHEIM  RUE DU TEMPLE RIXHEIM': (47.7465772887927, 7.395697508693975,'MULHOUSE'),
 'GYMNASE JEAN-PIERRE ADAMS  RUE SAINT EXUPERY THIONVILLE': (49.368792001232244, 6.1617871693498225,'THIONVILLE'),
 'CHARLES PERRAULT 2  RUE CHARLES PERRAULT EPINAL': (48.18482227012505, 6.457693540478761,'EPINAL'),
 'CHARLES ROTH  RUE JACQUES ANQUETIL DIEULOUARD': (48.84118446985741, 6.059275127005468,'BASSIN MUSSIPONTIN'),
 'GYMNASE DU TECHNOPOLE 5  BOULEVARD ARAGO METZ': (49.10206046306156, 6.221777042356304,'METZ'),
 'SALLE DES SPORTS  ESPLANADE JEAN COLBERT TAISSY': (49.2154326887296, 4.080864354965495,'TAISSY'),
 'GYMNASE DES MALTERIES 49  RUE DES MALTERIES SCHILTIGHEIM': (48.60461444093289, 7.743636698162694,'STRASBOURG'),
 'ANATOLE DE MARNEFFE  ALLEE DES CHALETS HERSERANGE': (49.521120775731156, 5.788602384697453,'P2H'),
 'RAYMOND LAURENT  PLACE CHARLES DE GAULLE ROSIERES PRES TROYES': (48.257768069543744, 4.0648065981523995,'ROSIERES'),
 'PALAIS DES SPORTS  AVENUE DE SAINT DIE EPINAL': (48.1866131980697, 6.470802440478821,'EPINAL'),
 'COMPLEXE SPORTIF  RUE DE LA FORET NOIRE RIXHEIM': (47.76173567627065, 7.395315665557758,'MULHOUSE'),
 'GYMNASE MUNICIPAL 19  RUE ALBERT LEBRUN LEXY': (49.498149801096545, 5.73181297335421,'P2H'),
 'LA NOUE LUTEL  RUE DE LA MALADIERE STE SAVINE': (48.294719768310166, 4.0262422435126775,'ROSIERES'),
 'COMPLEXE SPORTIF ST SYMPHORIEN  BOULEVARD ST SYMPHORIEN LONGEVILLE LES METZ': (49.11298982084647, 6.16174765769966,'METZ'),
 'GYMNASE DU 8 MAI 1945  PLACE DU 8 MAI BLENOD LES PONT A MOUSSON': (48.87976908832598, 6.052105000792116,'BASSIN MUSSIPONTIN'),
 'GYMNASE MUNICIPAL FERNAND GANNE  RUE FERNAND GANNE ST JULIEN LES VILLAS': (48.27311547277948, 4.09719497910932,'ROSIERES')}

In [21]:
Gymnases = Gymnases_2023
x = [Gymnases[d][1] for d in Gymnases]
y = [Gymnases[d][0] for d in Gymnases]
label = [Gymnases[d][2] for d in Gymnases]

### Using Folium

In [22]:
df2.index

Int64Index([  5,   8,  12,  18,  29,  32,  39,  47,  52,  58,  65,  68,  73,
             78,  85,  92,  99, 106, 113, 119, 124, 131],
           dtype='int64')

### Tracé des match d'EPINAL

In [23]:
for j in df2.index:
    try: 
        # print(df2.loc[j,'lieu'],Gymnases_2023[df2.loc[j,'lieu']])
        df2.loc[j,'lat'] = Gymnases_2023[df2.loc[j,'lieu']][0]
        df2.loc[j,'lon'] = Gymnases_2023[df2.loc[j,'lieu']][1]
    except KeyError: print('Lieu non défini')

Lieu non défini
Lieu non défini
Lieu non défini
Lieu non défini
Lieu non défini


In [24]:
list(df2['lieu'].unique())

['MONTAGNE VERTE  PLACE DE LA MONTAGNE VERTE COLMAR',
 'CHARLES ROTH  RUE JACQUES ANQUETIL DIEULOUARD',
 'PALAIS DES SPORTS  AVENUE DE SAINT DIE EPINAL',
 'GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY',
 'ANDRE ERBLAND 50  BD PRESIDENT ROOSEVELT MULHOUSE',
 'CENTRE SPORTIF INTERCOMMUNAL EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT',
 'GYMNASE JEAN-PIERRE ADAMS  RUE SAINT EXUPERY THIONVILLE',
 "GYMNASE DE L'EUROPE  ALLES D'ATHENES SCHILTIGHEIM",
 '— — —',
 'COMPLEXE SPORTIF DE HOCHFELDEN 1  RUE DES 4 VENTS HOCHFELDEN']

In [25]:
message_start = f"""
          <head>
          <meta http-equiv="Content-Type" content="text/html; charset=latin-1">
          <title>Lieu des Matchs</title>"""
message_style = """
          <style type="text/css" media="screen">
            #customers {
              font-family: "Trebuchet MS", Arial, Helvetica, sans-serif;
              font-size: 14px;
              border-collapse: collapse;
              width: 100%;
            }
            #customers td, #customers th {
              border: 1px solid #ddd;
              padding: 8px;
            }
            #customers tr:nth-child(even){background-color: #f2f2f2;}
            #customers tr:hover {background-color: #ddd;}
            #customers th {
              padding-top: 12px;
              padding-bottom: 12px;
              text-align: left;
              background-color: #003d77;
              color: white;
            }
          </style>
        </head>
        <body>
        """
titre = f'''
        <h3 align="center" style="font-size:16px font-family:Arial"><b>Journées de Championnat de l'équipe {Equipe}</b></h3>
        '''

In [26]:
lat_center = (df2.lat.max()+df2.lat.min())/2
lon_center = (df2.lon.max()+df2.lon.min())/2
m = folium.Map(location=[lat_center, lon_center], zoom_start=8,width='100%',height='100%')
numero=''
for j in list(df2['lieu'].unique()):
    if j == '— — —': continue
    else:
        print(j)
        df3 = df2[df2['lieu']==j]
        # display(df3)
        message_body = df3[['Journee','Date','equipe_dom','equipe_ext']].to_html(index=False, table_id="customers") #set table_id to your css style name
        message_end = """</body>"""
        popContent = (message_start +titre+ message_style + message_body + message_end)

        iframe = folium.IFrame(popContent)
        popup1 = folium.Popup(iframe,
                                      min_width=800,
                                      max_width=1100)

        numero = ''
        for j in df3['Journee'].unique():
            numero += str(j)+' '
        folium.Marker(
                location=[df3['lat'].unique()[0], df3['lon'].unique()[0]],
                popup=popup1,
                icon=folium.DivIcon(icon_size=(30,75), icon_anchor=(15,15), html=f'<div style="font-size: 14pt; color : blue;text-align:center";>{numero}</div>'),
        ).add_to(m)
        m.add_child(folium.CircleMarker([df3['lat'].unique()[0], df3['lon'].unique()[0]], radius=15))
m.get_root().html.add_child(folium.Element(titre))
m.save("./ANALYSE/Championnat_U18.html")
m

MONTAGNE VERTE  PLACE DE LA MONTAGNE VERTE COLMAR
CHARLES ROTH  RUE JACQUES ANQUETIL DIEULOUARD
PALAIS DES SPORTS  AVENUE DE SAINT DIE EPINAL
GYMNASE PROVENCAL 69 QUAI RENE 2 NANCY
ANDRE ERBLAND 50  BD PRESIDENT ROOSEVELT MULHOUSE
CENTRE SPORTIF INTERCOMMUNAL EUGENE GRIESMAR  QUAI ALBRECHT SELESTAT
GYMNASE JEAN-PIERRE ADAMS  RUE SAINT EXUPERY THIONVILLE
GYMNASE DE L'EUROPE  ALLES D'ATHENES SCHILTIGHEIM
COMPLEXE SPORTIF DE HOCHFELDEN 1  RUE DES 4 VENTS HOCHFELDEN


### Localisation de tous les Gymnases

In [19]:
lat_center = (max(y)+min(y))/2
lon_center = (max(x)+min(x))/2
titre = f'''<h3 align="center" style="font-size:16px font-family:Arial"><b>Gymnases du Championnat Handball U18M Saison 2022-23</b></h3>'''
m = folium.Map(location=[lat_center, lon_center], zoom_start=8)
eq=[]
for i, txt in enumerate(label):
    if txt not in eq:
        folium.Marker(
            location=[y[i], x[i]],
            popup=txt
        ).add_to(m)
        eq.append(txt)
m.get_root().html.add_child(folium.Element(titre))
m.save("./ANALYSE/Gymnases.html")
m

In [49]:
lat_moy = np.mean(np.array(y))

48.467259284495746

In [56]:
max(y)

49.36877802825876

In [57]:
min(y)

47.753907195499984

# Recherche les coordonnées à partir de LatLon website

In [31]:
# Recherche les coordonnées à partir du site LatLon
def extract_coordinates(add):
    # install chrome web driver https://sites.google.com/a/chromium.org/chromedriver/downloads
    s = Service("C:\Program Files\chromedriver.exe")
    driver = webdriver.Chrome(service=s)
    # Pointing to the targeted poule
    driver.get("https://www.latlong.net/")
    time.sleep(1)
    # Close the window that is opening when opening from chrome machine
    Path = '//*[@id="onetrust-accept-btn-handler"]'
    driver.find_element(By.XPATH, Path).click()
    time.sleep(1)
    #
    inputName = driver.find_element(By.ID, "place")
    inputName.send_keys(add)
    driver.find_element(By.ID, "btnfind").click()
    time.sleep(1)
    lat = driver.find_element(By.ID,"lat")
    print(add,lat.text)
    driver.close()
    return lat.text

In [32]:
# extract_coordinates(adresses[0])

# Recherche les coordonnées à partir de Google, est bloqué par le site

In [33]:
def extract_coordinates(add):
    #print(add)
    url = "https://www.google.fr/maps/dir/"+add.replace(' ','+')
    #url = "https://www.google.fr/maps/dir/5+Rue+d'Usson+du+Poitou,+57730+Folschviller"
    #print(url)
    request_url = urllib.request.urlopen(url) 
    data = request_url.read()
    data = data.decode('UTF-8')
    try:
        coord =re.findall('\[null,null,(\d+\.\d+,\d+\.\d+)]',data)
    except:
        coord = ['non trouvé']
    print(coord[0])
    time.sleep(5)
    return coord[0]

In [34]:
# extract_coordinates(adresses[0])